In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [ ]:
dataset, dataset_info = tfds.load('malaria', with_info=True, shuffle_files=True,as_supervised=True, split='train')


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
dataset

In [ ]:
dataset_info

In [ ]:
for data in dataset.take(1):
  print(data)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  DATASET_SIZE = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))
  return train_dataset, val_dataset, test_dataset


In [ ]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

train_dataset, val_dataset, test_dataset = splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO )


In [ ]:
for i,(image,label) in enumerate(train_dataset.take(16)):
  ax=plt.subplot(4,4,i+1)
  plt.imshow(image)
  plt.title(dataset_info.features['label'].int2str(label))
  plt.axis("off")

In [ ]:
dataset_info.features['label'].int2str(0)

In [ ]:
dataset_info.features['label'].int2str(1)

# data processing


In [20]:
def resize_rescale(image, label):
    # Print the shape of the image tensor for debugging
    print("Original image shape:", image.shape)

    # Ensure the image has 4 dimensions (batch, height, width, channels)
    if len(image.shape) == 3:
        image = tf.expand_dims(image, axis=0)  # Add batch dimension if missing
        print("Image shape after adding batch dimension:", image.shape)
    elif len(image.shape) != 4:
        raise ValueError('\'images\' must have either 3 or 4 dimensions.')

    # Resize and normalize image
    image = tf.image.resize(image, [img_size, img_size])
    image = image / 255.0
    return image[0], label  # Remove batch dimension for returning

# Apply preprocessing to datasets
train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)

# Batch the datasets
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

Original image shape: (None, None, 3)
Image shape after adding batch dimension: (1, None, None, 3)
Original image shape: (None, None, 3)
Image shape after adding batch dimension: (1, None, None, 3)


In [13]:
for (image,label) in train_dataset.take(1):
  print(image,label)

tf.Tensor(
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], shape=(145, 148, 3), dtype=uint8) tf.Tensor(1, shape=(), dtype=int64)


model **building**


In [21]:
img_size=244
model = tf.keras.Sequential([
    layers.InputLayer(input_shape=(img_size, img_size, 3)),
    layers.Conv2D(filters=6, kernel_size=5, padding='valid', activation='relu', strides=1),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=2, strides=1),
    layers.Conv2D(filters=16, kernel_size=5, padding='valid', activation='relu', strides=1),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=2, strides=1),
    layers.Flatten(),
    layers.Dense(100, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(10, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(1, activation='sigmoid')
])

model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 240, 240, 6)         │             456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 240, 240, 6)         │              24 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 239, 239, 6)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 235, 235, 16)        │           2,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 235, 235, 16)        │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 234, 234, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 876096)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │      87,609,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 100)                 │             400 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 10)                  │           1,010 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 10)                  │              40 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 87,614,121 (334.22 MB)

 Trainable params: 87,613,857 (334.22 MB)

 Non-trainable params: 264 (1.03 KB)

In [22]:
model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss = BinaryCrossentropy(),
    metrics=['accuracy']
)

In [17]:
train_dataset

<_TakeDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [18]:
val_dataset

<_TakeDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [23]:
model.fit(train_dataset,validation_data=val_dataset,epochs=15,verbose=1)

Epoch 1/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 87s 107ms/step - accuracy: 0.6478 - loss: 0.6405 - val_accuracy: 0.6065 - val_loss: 0.6395
Epoch 2/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 66s 96ms/step - accuracy: 0.7398 - loss: 0.5216 - val_accuracy: 0.8904 - val_loss: 0.3043
Epoch 3/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 61s 88ms/step - accuracy: 0.9142 - loss: 0.2352 - val_accuracy: 0.5561 - val_loss: 0.7413
Epoch 4/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 61s 89ms/step - accuracy: 0.9320 - loss: 0.1919 - val_accuracy: 0.8682 - val_loss: 0.3817
Epoch 5/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 61s 88ms/step - accuracy: 0.9431 - loss: 0.1594 - val_accuracy: 0.8748 - val_loss: 0.4122
Epoch 6/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 66s 96ms/step - accuracy: 0.9445 - loss: 0.1497 - val_accuracy: 0.9114 - val_loss: 0.3313
Epoch 7/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 77s 89ms/step - accuracy: 0.9573 - loss: 0.1159 - val_accuracy: 0.9027 - val_loss: 0.4300
Epoch 8/15
689/689 ━━━━━━━━━━━━━━━━━━━━ 65s 95ms/step - accuracy: 0.9641 - loss: 0.0961 -

In [24]:
model.save("malaria_detection_model.keras")